# 1. 环境配置

## 1.1 python 环境准备

In [ ]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. 回调（Callback）

## 2.1 简介

Callback 本质是一个Hook 机制，当 Chain / Model / Tool / Retriever 执行时，自动触发事件（start/end/error/token）。我们可以把 Callback 系统理解成三层结构：

- Run（执行生命周期）：是 Callback 系统中的“生命周期抽象”，不是一个类，而是执行引擎内部的概念。任何一次执行，都会触发一套事件。
- Handler（事件处理器）：是决定事件发生后要做什么
- Manager（调度器）：假如我们同时创建了多个 Handler，并希望同时运行，那我们可以通过 Manager 这个容器将其进行组合。

## 2.2 代码示例

下面就让我用一个简单的例子带大家看看如何实现回调。在演示之前，我们需要准备一段简单的 LCEL 代码作为回调的对象：

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import os

prompt = ChatPromptTemplate.from_messages([
  ("system", "你是一个简洁的中文助手。"),
  ("user", "{question}")])

model = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

parser = StrOutputParser()

chain = prompt | model | parser

然后我们就可以基于 BaseCallbackHandler 类来定义一些具体的方法了。
这里我们定义了在 chain 的开始和结束以及 llm 的开始、结束和报错时候显示的信息：

In [3]:
from langchain_core.callbacks import BaseCallbackHandler

class MyCustomCallbackHandler(BaseCallbackHandler):
  def on_chain_start(self, serialized, inputs, **kwargs):
    print("🚀 Chain 开始执行了！")
    print(f"📥 输入内容：{inputs}")

  def on_chain_end(self, outputs, **kwargs):
    print("✅ Chain 执行结束！")
    print(f"📤 输出内容：{outputs}")

  def on_llm_start(self, serialized, prompts, **kwargs):
    print("🤖 LLM 开始生成响应...")
    print(f"📝 Prompt 内容：{prompts}")

  def on_llm_end(self, response, **kwargs):
    print("🎉 LLM 响应生成完成！")
    print(f"🧾 原始响应：{response}")

  def on_llm_error(self, error, **kwargs):
    print("❌ LLM 出现错误：", error)

接下来我们就可以将其制作成 handler 并在 .invoke 调用时载入进去：

In [4]:
handler = MyCustomCallbackHandler()

result = chain.invoke(
  {"question": "用一句话介绍一下 LangSmith 是干什么的？"},
  config={"callbacks": [handler]}
)

print("\n最终输出：", result)

🚀 Chain 开始执行了！
📥 输入内容：{'question': '用一句话介绍一下 LangSmith 是干什么的？'}
🚀 Chain 开始执行了！
📥 输入内容：{'question': '用一句话介绍一下 LangSmith 是干什么的？'}
✅ Chain 执行结束！
📤 输出内容：messages=[SystemMessage(content='你是一个简洁的中文助手。', additional_kwargs={}, response_metadata={}), HumanMessage(content='用一句话介绍一下 LangSmith 是干什么的？', additional_kwargs={}, response_metadata={})]
🤖 LLM 开始生成响应...
📝 Prompt 内容：['System: 你是一个简洁的中文助手。\nHuman: 用一句话介绍一下 LangSmith 是干什么的？']
🎉 LLM 响应生成完成！
🧾 原始响应：generations=[[ChatGeneration(text='LangSmith 是一个用于构建、测试和部署语言模型应用的开发平台。', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='LangSmith 是一个用于构建、测试和部署语言模型应用的开发平台。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 14, 'total_tokens': 28, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ernie-3.5-8k', 'system_fingerprint': None, 'id': 'as-a7ppf8dnvf', 'finish_reason': 'stop', 'logprobs': None}

这里我们发现一个问题，就是 chain 开始连续执行了两次，然后执行结束了也有两次。其实两次 on_chain_start 和两次 on_chain_end 回调不是重复打印的 bug，而是 LangChain 链式结构 + LCEL 的运行机制所致。

第一次的时候，其实触发了整体链 prompt | llm | parser 的外层大链。而第二次的时候其实是触发了内部子链 prompt | llm 。而连续结束的也是一样的，我们是触发了节点parse。第二次则是整体大链的结束。

假如我们不想要看到子链的调用过程，只关注主链的话，我们就可以加入一个 parent_run_id 的判定，假如是 None 的时候，那就代表说其是主链，这个时候才打印就可以只在主链展示了：

In [5]:
class MyCustomCallbackHandler(BaseCallbackHandler):
  def on_chain_start(self, serialized, inputs, parent_run_id=None, **kwargs):
    if parent_run_id is None:
      print("🚀 Chain 开始执行了！")
      print(f"📥 输入内容：{inputs}")

  def on_chain_end(self, outputs, parent_run_id=None, **kwargs):
    if parent_run_id is None:
      print("✅ Chain 执行结束！")
      print(f"📤 输出内容：{outputs}")

  def on_llm_start(self, serialized, prompts, **kwargs):
    print("🤖 LLM 开始生成响应...")
    print(f"📝 Prompt 内容：{prompts}")

  def on_llm_end(self, response, **kwargs):
    print("🎉 LLM 响应生成完成！")
    print(f"🧾 原始响应：{response}")

  def on_llm_error(self, error, **kwargs):
    print("❌ LLM 出现错误：", error)

当然我们也可以传入多个 Callback，比如再制定另外一个 LoggingCallbackHandler：

In [6]:
class LoggingCallbackHandler(BaseCallbackHandler):
    def on_llm_end(self, response, **kwargs):
        print("📘 记录日志：模型已完成响应")

# 多个 handler 
my_handler_1 = MyCustomCallbackHandler() 
my_handler_2 = LoggingCallbackHandler()

那在调用时我们可以把这两个 handler 分别添加进去，其会按传入顺序进行执行：

In [7]:
result = chain.invoke(
  {"question": "用一句话介绍一下 LangSmith 是干什么的？"},
  config={"callbacks": [my_handler_1, my_handler_2]} 
)

print("\n最终输出：", result)

🚀 Chain 开始执行了！
📥 输入内容：{'question': '用一句话介绍一下 LangSmith 是干什么的？'}
🤖 LLM 开始生成响应...
📝 Prompt 内容：['System: 你是一个简洁的中文助手。\nHuman: 用一句话介绍一下 LangSmith 是干什么的？']
🎉 LLM 响应生成完成！
🧾 原始响应：generations=[[ChatGeneration(text='LangSmith 是一个用于构建、测试和评估语言模型应用的综合性开发平台。', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='LangSmith 是一个用于构建、测试和评估语言模型应用的综合性开发平台。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 14, 'total_tokens': 29, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ernie-3.5-8k', 'system_fingerprint': None, 'id': 'as-f1t7vcikq6', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2261-0a67-7020-a737-319e5ab41f81-0', usage_metadata={'input_tokens': 14, 'output_tokens': 15, 'total_tokens': 29, 'input_token_details': {}, 'output_token_details': {}}))]] llm_output={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 14, 't

当然我们也可以通过 CallbackManager 来进行多 handler 的管理：

In [8]:
from langchain_core.callbacks import CallbackManager

manager = CallbackManager([my_handler_1, my_handler_2])

那在调用时我们可以把这两个 handler 分别添加进去，其会按传入顺序进行执行：

In [9]:
result = chain.invoke(
  {"question": "用一句话介绍一下 LangSmith 是干什么的？"},
  config={"callbacks": manager.handlers} 
)

print("\n最终输出：", result)

🚀 Chain 开始执行了！
📥 输入内容：{'question': '用一句话介绍一下 LangSmith 是干什么的？'}
🤖 LLM 开始生成响应...
📝 Prompt 内容：['System: 你是一个简洁的中文助手。\nHuman: 用一句话介绍一下 LangSmith 是干什么的？']
🎉 LLM 响应生成完成！
🧾 原始响应：generations=[[ChatGeneration(text='LangSmith 是一个用于构建、测试和调试语言应用（如大语言模型应用）的综合性平台。', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='LangSmith 是一个用于构建、测试和调试语言应用（如大语言模型应用）的综合性平台。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ernie-3.5-8k', 'system_fingerprint': None, 'id': 'as-givaqcvgzi', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2261-13c1-7791-93ba-cfce90959ff1-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {}, 'output_token_details': {}}))]] llm_output={'token_usage': {'completion_tokens': 21, 'prompt_tok

## 2.3 智能体调用示例

对于旧版 LangChain 的智能体调用，默认在终端就会打印相关回调的内容：

### 2.3.1 创建大模型

In [10]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
 model="ernie-4.5-turbo-128k",
 openai_api_key=os.environ.get("OPENAI_API_KEY"),
 base_url="https://aistudio.baidu.com/llm/lmapi/v3" 
)

### 2.3.2 创建工具

In [11]:
from langchain.tools import tool
import re

@tool
def calculate(what: str) -> str:
  """
    Use this tool to perform arithmetic calculations.

    Input requirements:
    - The input must be a single-line arithmetic expression
    - Use only numbers and operators: + - * / ( )
    - DO NOT include any text, explanations, markdown, or words like "Observation"

    Examples:
    - 37 + 20
    - 57 * 2
    - (4 * 7) / 3

    This tool returns the numeric result as a string.
  """
  # 1️⃣ 去掉 Markdown
  text = what.replace("```", "").strip()
  # 2️⃣ 只取第一行（🔴 核心）
  first_line = text.splitlines()[0]
  # 3️⃣ 只允许算术字符（🔒 再保险）
  first_line = re.sub(r"[^0-9\.\+\-\*\/\(\) ]", "", first_line)
  return str(eval(first_line))

@tool
def average_dog_weight(name: str) -> str:
  """
    Use this tool to look up the average adult weight of a dog breed.

    Input requirements:
    - The input should ONLY be the dog breed name
    - Do NOT include extra text, questions, or explanations

    Examples:
    - Border Collie
    - Scottish Terrier
    - Toy Poodle

    This tool returns a short sentence stating the average weight in pounds (lbs).
  """
  name = name.lower()
  if "scottish terrier" in name:
    return "Scottish Terriers average 20 lbs"
  elif "border collie" in name:
    return "A Border Collie's average weight is 37 lbs"
  elif "toy poodle" in name:
    return "A Toy Poodle's average weight is 7 lbs"
  else:
    return "An average dog weighs 50 lbs"

tools = [calculate, average_dog_weight]

### 2.3.3 创建提示词

In [12]:
from langchain_classic import hub
prompt = hub.pull("hwchase17/react-chat")

### 2.3.4 搭建 Agent 框架

In [13]:
from langchain_classic.agents import AgentExecutor, create_react_agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### 2.3.5 添加记忆

In [14]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

store = {}
def get_session_history(session_id: str):
 if session_id not in store:
  store[session_id] = InMemoryChatMessageHistory()
 return store[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
  agent_executor,
  get_session_history=get_session_history,
  input_messages_key="input",
  history_messages_key="chat_history")

### 2.3.6 实际调用

In [15]:
config = {"configurable": {"session_id": "test-session"}}

print(agent_with_chat_history.invoke({"input": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight?"}, config)["output"])

print(agent_with_chat_history.invoke({"input": "how about double it?"}, config)["output"])



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: average_dog_weight
Action Input: Border Collie
ObservationA Border Collie's average weight is 37 lbsThought: Do I need to use a tool? Yes
Action: average_dog_weight
Action Input: Scottish Terrier
ObservationScottish Terriers average 20 lbsThought: Do I need to use a tool? Yes
Action: calculate
Action Input: 37 + 20
Observation57Thought: Do I need to use a tool? No
Final Answer: The combined weight of your Border Collie and Scottish Terrier is 57 lbs.

> Finished chain.
The combined weight of your Border Collie and Scottish Terrier is 57 lbs.


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: calculate
Action Input: 57 * 2
Observation114```
Thought: Do I need to use a tool? No
Final Answer: The combined weight of your dogs doubled would be 114 lbs.
```

> Finished chain.
The combined weight of your dogs doubled would be 114 lbs.
```


## 2.4 ConversationChain 调用示例
对于旧版 ConversationChain + ConversationBufferMemory 同样可以看到整个回调的全过程：

In [16]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import ConversationBufferMemory
import os
llm = ChatOpenAI(
model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, 
  memory=memory,
  verbose=True)
response_1 = conversation.predict(input="你好，我叫李剑锋")
print("对话 1:", response_1) 
response_2 = conversation.predict(input="请问1+1等于多少?")
print("对话 2:", response_2)
response_3 = conversation.predict(input="我的名字是?")
print("对话 3:", response_3) 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我叫李剑锋
AI:


C:\Users\76391\AppData\Local\Temp\ipykernel_14032\4291774188.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\76391\AppData\Local\Temp\ipykernel_14032\4291774188.py:10: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use `langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(llm=llm,



> Finished chain.
对话 1: 你好呀，李剑锋！很高兴认识你～你的名字听起来很有力量感呢，"剑锋"是不是有"锐利的剑刃"的意思呀？感觉像武侠小说里的侠客名字！你平时喜欢做什么有趣的事情吗？比如运动、看书，或者探索新科技？😊


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我叫李剑锋
AI: 你好呀，李剑锋！很高兴认识你～你的名字听起来很有力量感呢，"剑锋"是不是有"锐利的剑刃"的意思呀？感觉像武侠小说里的侠客名字！你平时喜欢做什么有趣的事情吗？比如运动、看书，或者探索新科技？😊
Human: 请问1+1等于多少?
AI:

> Finished chain.
对话 2: 这可真是个经典问题呢！1+1在基础的数学运算里，答案就是2啦。不过要是跳出数学，放到不同情境里，答案可就五花八门啦，像两个东西合到一起、两个人携手合作之类的。你心里是不是还藏着别的有趣想法呀？


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not kn